In [1]:
# Installing Required modules 
!pip install loguru
!pip install matplotlib
!pip install numpy
!pip install pathlib2
!pip install pandas
!pip install saxpy

In [2]:
#Importing all modules
import sys
import time
import os

import pandas as pd

from loguru import logger
from pathlib import Path

from Naive.node import Node

In [3]:
# Load Datasets ECG Data set
time_series_data = pd.read_csv('Dataset/Sales_Transaction_Dataset_Weekly_Final.csv')
time_series_data

,Product_Code,W0,W1,W2,W3,W4,W5,W6,W7,W8,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
0,P1,11,12,10,8,13,12,14,21,6,...,4,7,8,10,12,3,7,6,5,10
1,P2,7,6,3,2,7,1,6,3,3,...,2,4,5,1,1,4,5,1,6,0
2,P3,7,11,8,9,10,8,7,13,12,...,6,14,5,5,7,8,14,8,8,7
3,P4,12,8,13,5,9,6,9,13,13,...,9,10,3,4,6,8,14,8,7,8
4,P5,8,5,13,11,6,7,9,14,9,...,7,11,7,12,6,6,5,11,8,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,P815,0,0,1,0,0,2,1,0,0,...,0,1,1,0,0,1,0,0,2,0
807,P816,0,1,0,0,1,2,2,6,0,...,3,3,4,2,4,5,5,5,6,5
808,P817,1,0,0,0,1,1,2,1,1,...,2,0,0,2,2,0,0,0,4,3
809,P818,0,0,0,1,0,0,0,0,1,...,0,0,0,1,1,0,0,0,2,0


In [4]:
columns = list(time_series_data.columns)
time_series_index = columns.pop(0)  # remove product code
columns

['W0',
 'W1',
 'W2',
 'W3',
 'W4',
 'W5',
 'W6',
 'W7',
 'W8',
 'W9',
 'W10',
 'W11',
 'W12',
 'W13',
 'W14',
 'W15',
 'W16',
 'W17',
 'W18',
 'W19',
 'W20',
 'W21',
 'W22',
 'W23',
 'W24',
 'W25',
 'W26',
 'W27',
 'W28',
 'W29',
 'W30',
 'W31',
 'W32',
 'W33',
 'W34',
 'W35',
 'W36',
 'W37',
 'W38',
 'W39',
 'W40',
 'W41',
 'W42',
 'W43',
 'W44',
 'W45',
 'W46',
 'W47',
 'W48',
 'W49',
 'W50',
 'W51']

In [5]:
time_series_dict = dict()

for index, row in time_series_data.iterrows():
  time_series_dict[row[time_series_index]] = list(row[columns])

time_series_dict

{'P1': [11,
  12,
  10,
  8,
  13,
  12,
  14,
  21,
  6,
  14,
  11,
  14,
  16,
  9,
  9,
  9,
  14,
  9,
  3,
  12,
  5,
  11,
  7,
  12,
  5,
  9,
  7,
  10,
  5,
  11,
  7,
  10,
  12,
  6,
  5,
  14,
  10,
  9,
  12,
  17,
  7,
  11,
  4,
  7,
  8,
  10,
  12,
  3,
  7,
  6,
  5,
  10],
 'P2': [7,
  6,
  3,
  2,
  7,
  1,
  6,
  3,
  3,
  3,
  2,
  2,
  6,
  2,
  0,
  6,
  2,
  7,
  7,
  9,
  4,
  7,
  2,
  4,
  5,
  3,
  5,
  8,
  5,
  5,
  3,
  1,
  3,
  2,
  3,
  10,
  5,
  2,
  7,
  3,
  2,
  5,
  2,
  4,
  5,
  1,
  1,
  4,
  5,
  1,
  6,
  0],
 'P3': [7,
  11,
  8,
  9,
  10,
  8,
  7,
  13,
  12,
  6,
  14,
  9,
  4,
  7,
  12,
  8,
  7,
  11,
  10,
  7,
  7,
  13,
  11,
  8,
  10,
  8,
  14,
  5,
  3,
  13,
  11,
  9,
  7,
  8,
  7,
  9,
  6,
  12,
  12,
  9,
  3,
  5,
  6,
  14,
  5,
  5,
  7,
  8,
  14,
  8,
  8,
  7],
 'P4': [12,
  8,
  13,
  5,
  9,
  6,
  9,
  13,
  13,
  11,
  8,
  4,
  5,
  4,
  15,
  7,
  11,
  9,
  15,
  4,
  6,
  7,
  11,
  7,
  9,
  6,
  10,
  

In [12]:
# Implement Kapra Algorithm

def kapra(k_value, p_value, paa_value, dataset):
    
    max_level=4
    
    # ********* create-tree phase **********
    good_leaf_nodes = list()
    bad_leaf_nodes = list()
    
    print("Create-tree phase: start node splitting")
    node = Node(level=1, group=dataset, paa_value=paa_value)
    node.start_splitting(p_value, max_level, good_leaf_nodes, bad_leaf_nodes)
    print("Create-tree phase: finish node splitting")
    

    # ********** Recycle bad-leaves phase ********** 
    
    print("Start recycle bad-leaves phase")
    suppressed_nodes = list()
    if(len(bad_leaf_nodes) > 0):
        recycle_bad_leaf(p_value, good_leaf_nodes, bad_leaf_nodes, suppressed_nodes, paa_value)
    print("Finish recycle bad-leaves phase")
    suppressed_nodes_list = list()
        
    for node in suppressed_nodes:
        suppressed_nodes_list.append(node.group) 
        
        
    # ********  group formation phase **********
    print("Starting group formation phase")
    group_formation_phase()
    print("Finish group formation phase")
    

def recycle_bad_leaf(p_value, good_leaf_nodes, bad_leaf_nodes, suppressed_nodes, paa_value):
    
    bad_leaf_nodes_dict = dict()
    
    for node in bad_leaf_nodes:
        if node.level in bad_leaf_nodes_dict.keys():
            bad_leaf_nodes_dict[node.level].append(node)
        else:
            bad_leaf_nodes_dict[node.level] = [node]

        bad_leaf_nodes_size = sum([node.size for node in bad_leaf_nodes])
        
        if bad_leaf_nodes_size >= p:
            current_level = max(bad_leaf_nodes_dict.keys())
            
            
            while bad_leaf_nodes_size >= p:
                
                if current_level in bad_leaf_nodes_dict.keys():
                    merge_dict = dict()
                    keys_to_be_removed = list()
                    merge = False
                    
                    for current_level_node in bad_leaf_nodes_dict[current_level]:
                        pr_node = current_level_node.pattern_representation
                    
                        if pr_node in merge_dict.keys():
                            merge = True
                            
                            merge_dict[pr_node].append(current_level_node)
                            
                            if pr_node in keys_to_be_removed:
                                keys_to_be_removed.remove(pr_node)
                        else:
                            
                            merge_dict[pr_node] = [current_level_node]
                            
                            keys_to_be_removed.append(pr_node)
                    
                    
                    if merge:
                       
                        for k in keys_to_be_removed:
                            del merge_dict[k]
                        
                        for pr, node_list in merge_dict.items():
                            
                            group = dict()
                            
                            for node in node_list:
                                
                                bad_leaf_nodes_dict[current_level].remove(node)
                                
                                group.update(node.group)
                                
                            # check the current level
                            if current_level > 1:
                                level = current_level
                            else:
                                level = 1
                           
                            leaf_merge = Node(level=level, pattern_representation=pr,
                                group=group, paa_value=paa_value)

                            if leaf_merge.size >= p:
                                
                                leaf_merge.label = "good-leaf"
                                good_leaf_nodes.append(leaf_merge)
                                
                                bad_leaf_nodes_size -= leaf_merge.size
                            else: 
                                
                                leaf_merge.label = "bad-leaf"
                               
                                bad_leaf_nodes_dict[current_level].append(leaf_merge)

                
                temp_level = current_level-1 
                
                for node in bad_leaf_nodes_dict[current_level]: 
                   
                    if temp_level > 1:
                        
                        values_group = list(node.group.values())
                        
                        data = np.array(values_group[0])
                        
                        data_znorm = znorm(data)
                        
                        data_paa = paa(data_znorm, paa_value)
                        
                        pr = ts_to_string(data_paa, cuts_for_asize(temp_level))
                    else:
                        
                        pr = "a"*paa_value
                    
                    node.level = temp_level
                    
                    node.pattern_representation = pr

                
                if current_level > 0:
                   
                    if temp_level not in bad_leaf_nodes_dict.keys():
                        
                        bad_leaf_nodes_dict[temp_level] = bad_leaf_nodes_dict.pop(current_level)
                    else:
                       
                        bad_leaf_nodes_dict[temp_level] = bad_leaf_nodes_dict[temp_level] + bad_leaf_nodes_dict.pop(current_level) 
                    
                    current_level -= 1
                else:
                    break 
        
      
        remaining_bad_leaf_nodes = list(bad_leaf_nodes_dict.values())[0]
        for node in remaining_bad_leaf_nodes:
            suppressed_nodes.append(node)

    

def group_formation_phase():
    print ('AAAA')
       

        
if __name__ == "__main__":
    
    #Execute (k, P) Kapra algorithm
    start = time.time()
    
    # changes the parameters accordingly
    kapra(4, 2, 2, time_series_dict)
    
    end = time.time()
    
    Time_taken = round(float(end - start), 3)
    print('\n KAPRA algorithm Estimated Time taken: ' + str(Time_taken) + ' sec')
        

Create-tree phase: start node splitting


2022-04-20 23:33:03.402 | INFO     | Naive.node:start_splitting:89 - N can be split
2022-04-20 23:33:03.403 | INFO     | Naive.node:start_splitting:90 - Compute tentative good nodes and tentative bad nodes
2022-04-20 23:33:03.405 | INFO     | Naive.node:start_splitting:152 - Label all tb_node 0 as bad-leaf and split only tg_nodes 4
2022-04-20 23:33:03.958 | INFO     | Naive.node:start_splitting:89 - N can be split
2022-04-20 23:33:03.959 | INFO     | Naive.node:start_splitting:90 - Compute tentative good nodes and tentative bad nodes
2022-04-20 23:33:03.962 | INFO     | Naive.node:start_splitting:152 - Label all tb_node 0 as bad-leaf and split only tg_nodes 6
2022-04-20 23:33:04.261 | INFO     | Naive.node:start_splitting:89 - N can be split
2022-04-20 23:33:04.262 | INFO     | Naive.node:start_splitting:90 - Compute tentative good nodes and tentative bad nodes
2022-04-20 23:33:04.263 | INFO     | Naive.node:start_splitting:152 - Label all tb_node 0 as bad-leaf and split only tg_nodes 

Create-tree phase: finish node splitting
Start recycle bad-leaves phase
Finish recycle bad-leaves phase
Starting group formation phase
AAAA
Finish group formation phase

 KAPRA algorithm Estimated Time taken: 2.928 sec
